# Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction

*M. Zimmermann, Z. Abbas, K. Dzieciol and N. J. Shah, "Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction," in IEEE Transactions on Medical Imaging, vol. 37, no. 2, pp. 626-637, Feb. 2018, doi: 10.1109/TMI.2017.2771504.*

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys

sys.path.insert(0, "../src")

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import h5py
import ismrmrd
import matplotlib.pyplot as plt
import torch

from juart.conopt.functional.fourier import fourier_transform_adjoint
from juart.preproc.aux import process_siemens_folder

# from juart.preproc.aux import process_siemens_file
from juart.preproc.data import KSpaceData, get_shape
from juart.preproc.trajectory import KSpaceTrajectory

torch.set_num_threads(1)

In [2]:
# set folder to convert to h5d

# fname = "7T1026"
foldername = '/workspaces/juart/LocalData/'

In [3]:
# save = True
# pulseq = False

In [4]:
process_siemens_folder(foldername)

Siemens file is: /workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.dat
-----------------------------------------------------------------
Converting measurement 1 into file /workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5 in group dataset
-----------------------------------------------------------------
Using parameter map: IsmrmrdParameterMap_Siemens.xml
This file contains 1 measurement(s).
VD line file detected.
Protocol name [1]: CS_MP2RAGE_0.6mm_proposedTiming
Number of parameter buffers: 6
Buffer Name: Config
Buffer Name: Dicom
Buffer Name: Meas
Buffer Name: MeasYaps
Buffer Name: Phoenix
Buffer Name: Spice
Trajectory is: 1
Failed to find YAPS.lFirstFourierLine array
Failed to find YAPS.lFirstFourierPartition array
center_line = 180
center_partition = 144
Baseline: N4_VE12U_LATEST_20181126
Software version: syngo MR E12
Protocol name: CS_MP2RAGE_0.6mm_proposedTiming
Dwell time: 5400
Using parameter XSL: IsmrmrdP

ParcFileEntries[0].off_ = 10240
ParcFileEntries[0].len_ = 6360511584
siemens_dat.tellg() = 6360521632
Please check the result.


CompletedProcess(args=['siemens_to_ismrmrd', '-f', '/workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.dat', '-o', '/workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5'], returncode=0)

In [6]:
full_session_dir = foldername
h5_ismrmrd_fname = "meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5"
h5_preproc_fname = "preproc/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5"

In [7]:
full_h5_ismrmrd_fname = os.path.join(full_session_dir, h5_ismrmrd_fname)
full_h5_preproc_fname = os.path.join(full_session_dir, h5_preproc_fname)

In [8]:
print(full_h5_ismrmrd_fname)
print(full_h5_preproc_fname)

/workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5
/workspaces/juart/LocalData/preproc/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5


# Load Data

In [9]:
print("(1/2) Loading rawdata ...")

(1/2) Loading rawdata ...


In [10]:
dataset = ismrmrd.Dataset(
    full_h5_ismrmrd_fname,
    dataset_name="dataset",
    create_if_needed=False,
)

In [11]:
NCha, NCol, NLin, NPar, NSli, NSet, NEco = get_shape(dataset)

In [ ]:
# For now, manually set this to 160
# NPar = 160

In [12]:
print(NCha, NCol, NLin, NPar, NSli, NSet, NEco)

32 128 360 288 1 2 1


In [ ]:
# Conver to slice-by-slice problem

# fourier_transform_adjoint

In [ ]:
# NLin_retro = 8
# NPar_post, NSli_post = NSli, NPar
# NCha_comp, ISet_comp, IEco_comp = 8, slice(NSet - 1, NSet), slice(0, 1)
# NImx, NImy, ISet_coil, IEco_coil = 256, 256, slice(15, 19), slice(0, 1)
# NAcl, NUsf = 32, 2

In [13]:
kdata = torch.zeros((NCha, NCol, NLin, NPar, NSli, NSet, NEco), dtype=torch.complex64)

In [ ]:
for index in range(dataset.number_of_acquisitions()):

    # Read the acquisition.
    acquisition = dataset.read_acquisition(index)

    
    if acquisition.data.shape == kdata.shape[:2]:
        # Update shared memory.
        kdata[
            :,
            :,
            acquisition.idx.kspace_encode_step_1,
            acquisition.idx.kspace_encode_step_2,
            acquisition.idx.slice,
            acquisition.idx.set,
            acquisition.idx.contrast,
        ] = torch.tensor(acquisition.data)
    else:
        print(index, acquisition.data.shape)

0 (32, 128)
1 (32, 128)
2 (32, 128)
3 (32, 128)
4 (32, 128)
5 (32, 128)
6 (32, 128)
7 (32, 128)
8 (32, 128)
9 (32, 128)
10 (32, 128)
11 (32, 128)
12 (32, 128)
13 (32, 128)
14 (32, 128)
15 (32, 128)
16 (32, 128)
17 (32, 128)
18 (32, 128)
19 (32, 128)
20 (32, 128)
21 (32, 128)
22 (32, 128)
23 (32, 128)
24 (32, 128)
25 (32, 128)
26 (32, 128)
27 (32, 128)
28 (32, 128)
29 (32, 128)
30 (32, 128)
31 (32, 128)
32 (32, 128)
33 (32, 128)
34 (32, 128)
35 (32, 128)
36 (32, 128)
37 (32, 128)
38 (32, 128)
39 (32, 128)
40 (32, 128)
41 (32, 128)
42 (32, 128)
43 (32, 128)
44 (32, 128)
45 (32, 128)
46 (32, 128)
47 (32, 128)
48 (32, 128)
49 (32, 128)
50 (32, 128)
51 (32, 128)
52 (32, 128)
53 (32, 128)
54 (32, 128)
55 (32, 128)
56 (32, 128)
57 (32, 128)
58 (32, 128)
59 (32, 128)
60 (32, 128)
61 (32, 128)
62 (32, 128)
63 (32, 128)
64 (32, 128)
65 (32, 128)
66 (32, 128)
67 (32, 128)
68 (32, 128)
69 (32, 128)
70 (32, 128)
71 (32, 128)
72 (32, 128)
73 (32, 128)
74 (32, 128)
75 (32, 128)
76 (32, 128)
77 (32, 1

RuntimeError: The expanded size of the tensor (128) must match the existing size (608) at non-singleton dimension 1.  Target sizes: [32, 128].  Tensor sizes: [32, 608]

In [ ]:
kdata = KSpaceData((NCha, NCol, NLin, NPar, NSli, NSet, NEco))

In [ ]:
kdata.read_data(dataset, is_pulseq=False)

In [ ]:
# Undersample along line dimension
kdata.index_data([slice(None), slice(None), slice(NLin_retro), ...])

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# GRAPPA

In [ ]:
kdata.apply_mask(NUsf, NAcl, is_pulseq=True)

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

In [ ]:
kdata.reconstruct_partitions()

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# Coil compression

In [ ]:
# Swap Par and Sli Partition
kdata.swapaxes(3, 4)

In [ ]:
comp_matrix = kdata.compression_matrix(NCha_comp, ISet_comp, IEco_comp)

In [ ]:
kdata.compress_data(comp_matrix)

In [ ]:
# Swap Col and Lin dimension
kdata.swapaxes(1, 2)

In [ ]:
NCha, NLin, NCol, NPar, NSli, NSet, NEco = kdata.kdata_shape

In [ ]:
print("(4/4) Creating output file ....")

if save:
    with h5py.File(full_h5_preproc_fname, "w", libver="latest") as out_file:
        out_file.create_dataset(
            "d", data=kdata.kdata.numpy(), chunks=(NCha, NLin, NCol, 1, 1, 1, 1)
        )

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")

In [ ]:
vmax = 5e-4

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")

# Gradient Delay Correction (RING)

In [ ]:
ktraj = KSpaceTrajectory(NLin, NCol, NSet, NEco)

In [ ]:
S_corr = ktraj.estimate_gradient_delay(kdata.kdata)

In [ ]:
S_corr

In [ ]:
ktraj_corr = ktraj.correct_kspace_trajectory(S_corr)

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "k", data=ktraj_corr.numpy(), chunks=(2, NLin_retro, NCol, 1, 1, 1, 1)
        )

# Coil Sensitivity Estimation (SAKE + ESPIRiT)

In [ ]:
sensmaps = kdata.get_sensmaps(ktraj_corr, (NCha, NImx, NImy, 1, NSli, 1, 1))

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "C",
            data=sensmaps.numpy(),
            chunks=(NCha_comp, NImx, NImy, NPar_post, NSli_post, 1, 1),
        )

In [ ]:
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(torch.abs(sensmaps[0, :, :, 0, 80, 0, 0]).numpy())
plt.subplot(1, 3, 2)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0]).T.numpy())
plt.subplot(1, 3, 3)
plt.imshow(torch.abs(sensmaps[0, 128, :, 0, :, 0, 0]).T.numpy())

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(torch.abs(sensmaps[-1, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")